In [ ]:
include("optimization_library.jl");
include("mplstyle.jl");

# Exercise 7.1: Constraint Elimination

In [3]:
# Solving a problem with equality constraints Ax = b using constraint elimination and interior point method

# f: Objective function
# df: Gradient vector of the objective function
# Hf: Hessian matrix of the objective function
# g: Array of inequality constraints g_i(x) < 0
# dg: Array with the gradient of the inequality constraints
# Hg: Array with the Hessian of the inequality constraints
# A: matrix for equality constraints Ax = b
# b: vector for equality constraints Ax = b
# barrier_increase: Factor by how much to increase the barrier in every outer iteration
# maxiters: maximum number of outer iterations
# inner_maxiters: maximum number of inner iterations (within descent method)
# eps: Stopping criterion

function ConstraintElimination_ipm(f,df,Hf,A,b;g=[],dg=[],Hg=[],eps = 0.0001, barrier_increase = 2.0,
                                    maxiters = 100, inner_maxiters = 100)

    nx = size(A,2) # dimensions of the original problem
    nz = nx - LA.rank(A) # dimensions of the unconstrained problem
    
    # vectors for transformed positivity constraints --> entry gg[i] is a function depending on z (variable of the
    # problem where equality constraints are removed)
    
    ng = size(g,1)
    gg = Vector(undef, ng)
    dgg = Vector(undef, ng)
    Hgg = Vector(undef, ng)
       
    # trace of interior point method
    trace = 1
    
# ===============================================================================
    # 1. Find an initial admissible solution for, the equality constraints
    # Ax - b = 0
    # Ax = b
    x0 = A\b
    # 2. Construct the matrix F (see lecture 4 for the notation)
    F = LA.nullspace(A)
    # 3. Construct new objective functions, its gradient and Hessian 
    # Concerning the gradient and Hessian: Beware that you derive for z and have to respect the chain rule!
    
    obj(z) = f(F * z + x0)
    dobj(z) = F' * df(F * z + x0)
    Hobj(z) = F' * Hf(F * z + x0) * F
    
    # 4. Transform the inequality constraints, their gradients and Hessians to be functions of Fz + x0
    for i in 1:ng
        gg[i] = g[i](F * z + x0)
        dgg[i] = F' * dg[i](F * z + x0)
        Hgg[i] = F' * Hg[i](F * z + x0) * F
    end

    # 5. Randomly initialize z and find an admissible interior point for the transformed problem
    z0 = rand(nz)
    z_adm = ipm_admissible(z0; g=gg, dg=dgg, maxiters = 1000, eps=eps, inner_maxiters = 100,
                               barrier_increase = barrier_increase)
    
    # 6. use the admissible interior point to solve the transformed problem
    zstar, trace = ipm(obj, dobj, Hobj, z_adm, g = gg, dg = dgg, Hg = Hgg, barrier_increase = barrier_increase)
    
    
    # 7. reconstruct the solution of the original problem 
    xstar = F * zstar + x0
    
# ===============================================================================
    return xstar, trace
    
end

ConstraintElimination_ipm (generic function with 1 method)

# Exercise 7.2: Santa's Optimal Transport Problem

<img src="figures/santa-claus.png" width="300">

Solve Santa's optimal transport problem using your constraint elimination method implemented above.